In [1]:
import os
import json
import gc

import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [14]:
train_df = pd.read_csv('/app/AD_try/workshop_models/severstal-steel-defect-detection/train.csv')
# train_df['ImageId'] = train_df['ClassId'].apply(lambda x: x.split('_')[0])
# train_df['ClassId'] = train_df['ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

(7095, 4)


,ImageId,ClassId,EncodedPixels,hasMask
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,True
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...,True
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...,True
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...,True
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...,True


In [15]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(6666, 3)


,ImageId,ClassId,hasMask
6253,ef24da2ba.jpg,6,3
5740,db4867ee8.jpg,6,3
3435,84865746c.jpg,7,2
2203,5665ada1b.jpg,7,2
6149,eb44ad79d.jpg,7,2


In [17]:
sub_df = pd.read_csv('/app/AD_try/workshop_models/severstal-steel-defect-detection/sample_submission.csv')
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])
test_imgs.head()

,ImageId
0,0000f269f.jpg
1,000ccc2ac.jpg
2,002451917.jpg
3,003c5da97.jpg
4,0042e163f.jpg


In [18]:
non_missing_train_idx = mask_count_df[mask_count_df['hasMask'] > 0]
non_missing_train_idx.head()

,ImageId,ClassId,hasMask
6253,ef24da2ba.jpg,6,3
5740,db4867ee8.jpg,6,3
3435,84865746c.jpg,7,2
2203,5665ada1b.jpg,7,2
6149,eb44ad79d.jpg,7,2
